# AMR Quick Demo (sin Docker)

Este notebook te permite **probar YA** el flujo de detección de genes RAM **sin Docker**.

### Qué hace
1. Instala dependencias (Biopython, pandas, reportlab)
2. Carga una **secuencia query** (FASTA) y una **base de referencia** (FASTA con genes)
3. Alinea (pairwise) y calcula **identidad** y **cobertura** por gen
4. Filtra por umbrales (identidad ≥ 90%, cobertura ≥ 80%)
5. Muestra tabla, y genera **CSV** y **PDF**

> Nota: Este demo usa **alineamiento pairwise** (rápido y portable). Para alta escala/precisión usar BLAST+/RGI (lo dejamos listo en el proyecto grande).

In [ ]:
# 1) Instalar dependencias (ejecuta esta celda una vez) 
# Si ya las tienes, puedes saltarla.
import sys, subprocess
def pip_install(pkg):
    try:
        __import__(pkg.split("==")[0].split(">=")[0])
    except ImportError:
        print(f"Instalando {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for p in ["biopython>=1.81", "pandas>=2.0", "reportlab>=4.2", "ipywidgets>=8.0"]:
    pip_install(p)

print("Listo.")

In [ ]:
# 2) Imports clave
from Bio import SeqIO, pairwise2
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from ipywidgets import FileUpload, VBox, HBox, Button, FloatSlider, HTML
from IPython.display import display
import io, os, datetime

print("Imports OK")

In [ ]:
# 3) Widgets para cargar archivos (query y referencia)
#   - Sube tu archivo FASTA Query (ej: 'query.fasta')
#   - Sube tu FASTA de referencia con genes (encabezados con IDs de genes)
u_query = FileUpload(accept='.fa,.fasta', multiple=False)
u_ref = FileUpload(accept='.fa,.fasta', multiple=False)
btn_go = Button(description='Analizar', button_style='success')
thr_ident = FloatSlider(description='Identidad ≥', min=50, max=100, step=0.5, value=90)
thr_cov = FloatSlider(description='Cobertura ≥', min=50, max=100, step=0.5, value=80)
out_status = HTML(value='')
display(VBox([
    HTML('<h3>Sube tu FASTA Query</h3>'), u_query,
    HTML('<h3>Sube tu FASTA de referencia (genes)</h3>'), u_ref,
    HBox([thr_ident, thr_cov, btn_go]),
    out_status
]))

In [ ]:
# 4) Lógica de análisis (pairwise)
from Bio import SeqIO, pairwise2
import pandas as pd
import io

def fasta_from_upload(fu: FileUpload):
    if len(fu.value) == 0:
        return None
    k = list(fu.value.keys())[0]
    content = fu.value[k]['content']
    return io.StringIO(content.decode('utf-8', errors='ignore'))

def read_first_sequence(fh):
    # Toma la primera secuencia del FASTA (para demo). Puedes adaptar a varias.
    for rec in SeqIO.parse(fh, 'fasta'):
        return rec
    return None

def read_reference(fh):
    # Devuelve lista de (id, seq)
    refs = []
    for rec in SeqIO.parse(fh, 'fasta'):
        refs.append((rec.id, str(rec.seq)))
    return refs

def compute_identity_coverage(query_seq: str, ref_seq: str):
    # Alineamiento global simple; para genes más largos puedes probar local (pairwise2.localms)
    alignments = pairwise2.align.globalms(query_seq, ref_seq, 2, -1, -5, -0.5, one_alignment_only=True)
    if not alignments:
        return 0.0, 0.0
    aln = alignments[0]
    q_aln, r_aln, score, start, end = aln
    # Identidad = matches / largo alineado
    matches = sum(1 for a,b in zip(q_aln, r_aln) if a == b)
    length = len(q_aln)
    identity = 100.0 * matches / max(1, length)
    # Cobertura (sobre la query) = posiciones no-gap de la query en el alineamiento / len(query)
    cov = 100.0 * sum(1 for a in q_aln if a != '-') / max(1, len(query_seq))
    return identity, cov

results_df = pd.DataFrame()

def run_analysis(_):
    global results_df
    out_status.value = '<b>Procesando...</b>'
    qfh = fasta_from_upload(u_query)
    rfh = fasta_from_upload(u_ref)
    if not qfh or not rfh:
        out_status.value = '<span style="color:red">Faltan archivos FASTA.</span>'
        return
    qrec = read_first_sequence(qfh)
    refs = read_reference(rfh)
    if not qrec or not refs:
        out_status.value = '<span style="color:red">Revisa los FASTA (no se encontraron secuencias).</span>'
        return
    rows = []
    for rid, rseq in refs:
        ident, cov = compute_identity_coverage(str(qrec.seq), rseq)
        rows.append({
            'gene': rid,
            'identity': round(ident,2),
            'coverage': round(cov,2)
        })
    df = pd.DataFrame(rows).sort_values(['identity','coverage'], ascending=False)
    # Aplicar filtros
    df_f = df[(df.identity >= thr_ident.value) & (df.coverage >= thr_cov.value)].copy()
    results_df = df_f.reset_index(drop=True)
    out_status.value = f'<b>Listo.</b> {len(df_f)} hits tras filtrar.'
    display(df_f)

btn_go.on_click(run_analysis)
print('Listo. Pulsa ANALIZAR cuando subas los archivos.')

In [ ]:
# 5) Exportar CSV y PDF de resultados (ejecuta esto luego de ANALIZAR)
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
import os

out_dir = 'amr_outputs'
os.makedirs(out_dir, exist_ok=True)

if 'results_df' in globals() and not results_df.empty:
    # CSV
    csv_path = os.path.join(out_dir, 'results.csv')
    results_df.to_csv(csv_path, index=False)
    print('CSV guardado en:', csv_path)

    # PDF
    pdf_path = os.path.join(out_dir, 'report.pdf')
    c = canvas.Canvas(pdf_path, pagesize=A4)
    width, height = A4
    y = height - 2*cm
    c.setFont('Helvetica-Bold', 16)
    c.drawString(2*cm, y, 'Reporte RAM — Demo')
    y -= 1*cm
    c.setFont('Helvetica', 10)
    c.drawString(2*cm, y, f'Umbrales: identidad ≥ {thr_ident.value:.1f}%, cobertura ≥ {thr_cov.value:.1f}%')
    y -= 0.8*cm
    for _, row in results_df.head(40).iterrows():
        line = f"- {row['gene']} | ident={row['identity']}% | cov={row['coverage']}%"
        if y < 3*cm:
            c.showPage(); y = height - 2*cm; c.setFont('Helvetica', 10)
        c.drawString(2.2*cm, y, line)
        y -= 0.5*cm
    c.showPage(); c.save()
    print('PDF guardado en:', pdf_path)
else:
    print('Primero ejecuta el análisis y asegúrate de que haya resultados filtrados.')

### Sugerencias
- Si tu referencia es muy grande (miles de genes), este método puede tardar. Para eso es mejor el proyecto con **BLAST+/Celery**.
- Puedes cambiar la función de alineamiento por `pairwise2.localms` para alineamiento local.
- Este notebook sirve perfecto para **exponer** la idea y mostrar resultados sin instalar Docker.